In [1]:
!apt-get update
!apt-get install -y wget unzip libvulkan1
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -f -y
!apt-get install -y xvfb

!pip install selenium chromedriver-autoinstaller
!pip install bs4



Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,125 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd6

In [2]:
from selenium import webdriver
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import re


In [8]:
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

def scrolldown(driver, deep):
    for _ in range(deep):
        driver.execute_script('window.scrollBy(0,500)')
        time.sleep(0.1)

def get_page_data(driver, url):
    driver.get(url)
    scrolldown(driver, 50)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, "html.parser")

    table = soup.find('table', attrs={'id': 'global'}) or soup.find('table', attrs={'id': 'global-daily-totals'}) or soup.find('table')

    rows = []
    for row in table.find('tbody').find_all('tr'):
        cols = row.find_all('td')
        if len(cols) < 4:
            continue
        song_artist = cols[0].text.strip()
        if " - " in song_artist:
            artist, song = song_artist.split(" - ", 1)
        else:
            artist = ""
            song = song_artist
        days = cols[1].text.strip()
        T10 = cols[2].text.strip()
        Peak = cols[3].text.strip()
        pkx = cols[4].text.strip()
        PkStreams = cols[5].text.strip()
        Total = cols[6].text.strip()

        rows.append({"Artist": artist, "Song": song, "Days in top10": T10, "Peak": Peak, "Days on peak": pkx, "Days": days, "PkStreams": PkStreams, "Total": Total})

    df = pd.DataFrame(rows, columns=["Artist", "Song", "Days in top10", "Peak", "Days on peak", "Days", "PkStreams", "Total"])
    df.to_csv("spotify_songs.csv", index=False)

driver = webdriver.Chrome(options=chrome_options)
get_page_data(driver, "https://kworb.net/spotify/country/global_daily_totals.html")
driver.quit()

In [11]:
df = pd.read_csv("spotify_songs.csv")

def clean_days_on_peak(val):
    if pd.isna(val):
        return 1
    match = re.search(r'\d+', str(val))
    return int(match.group()) if match else 1

df['Days on peak'] = df['Days on peak'].apply(clean_days_on_peak)

df.to_csv('Scrap.csv', index=False)
df

,Artist,Song,Days in top10,Peak,Days on peak,Days,PkStreams,Total
0,The Weeknd,Blinding Lights,446.0,1,82,2169,"8,453,567","5,037,607,641"
1,Ed Sheeran,Shape of You,237.0,1,98,2689,"9,878,194","4,038,423,854"
2,Harry Styles,As It Was,426.0,1,81,1272,"16,103,849","3,999,274,385"
3,Lewis Capaldi,Someone You Loved,107.0,4,1,2371,"3,974,554","3,880,912,658"
4,Post Malone,Sunflower - Spider-Man: Into the Spider-Verse,213.0,1,16,2263,"5,033,261","3,554,729,660"
...,...,...,...,...,...,...,...,...
11452,Wisin,Adrenalina,NaN,188,1,1,"124,862","124,862"
11453,Iggy Azalea,Work,NaN,196,1,1,"120,954","120,954"
11454,Orup,Främling,NaN,199,1,1,"120,760","120,760"
11455,Ed Sheeran,Nina,NaN,199,1,1,"120,515","120,515"
